In [1]:
import cobra
import cplex
import optlang
import sys
import cobrakbase
sys.path.append("/Users/chenry/code/ModelSEEDpy")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager, FBAHelper, MSBuilder

cobrakbase 0.2.7


In [2]:
kbase_api = cobrakbase.KBaseAPI()
model = kbase_api.get_from_ws("Cjaponicus_Ecoli_Community",92637)
model.solver = 'optlang-cplex'
#Applying constraints based on KBase media
media = kbase_api.get_from_ws("ChitinM9Media",91858)
template = kbase_api.get_from_ws("GramNegModelTemplateV3","NewKBaseModelTemplates")
model = MSBuilder.gapfill_model(model,"bio1",template,media)

In [3]:
pkgmgr = MSPackageManager.get_pkg_mgr(model)
pkgmgr.getpkg("ElementUptakePkg").build_package({"C":60})
pkgmgr.getpkg("KBaseMediaPkg").build_package(media)

biomass_objective = model.problem.Objective(
    1 * model.reactions.bio1.flux_expression,
    direction='max')
model.objective = biomass_objective
    
sol=model.optimize()
model.summary()

In [4]:
#Applying community model kinetic constraints
pkgmgr.getpkg("CommKineticPkg").build_package(2100,{"1":0.5,"2":0.5})

with open('/Users/chenry/Dropbox/workspace/PNNLSFA/CommunityFBA/CommunityFBA.lp', 'w') as out:
    out.write(str(model.solver))

sol=model.optimize()
model.summary()

1:bio2
2:bio3
